In [53]:
import sympy as sp
from lbmpy.stencils import LBStencil, Stencil
from lbmpy.moments import MOMENT_SYMBOLS, moment_matrix
import numpy as np

# D2Q9

## Multiphase

In [54]:
d2q9 = LBStencil(Stencil.D2Q9)
Q = len(d2q9)
c = sp.Matrix(d2q9)
cs = sp.sqrt(sp.Rational(1,3))

x, y, z = MOMENT_SYMBOLS
one = sp.core.sympify(1)
c2 = x**2 + y**2
c4 = c2**2
moments = [
    one,
    x,
    y,
    3*c2-2,
    2*x**2-c2,
    x*y,
    (3*c2-4)*x,
    (3*c2-4)*y,
    9*c4-15*c2+2
]
M = moment_matrix(moments, stencil=d2q9)

In [55]:
kappa = sp.symbols('kappa')
pb = sp.symbols('p_0')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
u = sp.Matrix([ux,uy])
Drho2 = sp.Matrix([ [ sp.Derivative(rho,a)*sp.Derivative(rho,b) for b in [x,y] ] for a in [x,y] ])
D2rho = sum([ sp.Derivative(rho,a,2) for a in [x,y] ])
G = kappa*Drho2

one12 = sp.Rational(1,12)
one18 = sp.Rational(1,18)
one72 = sp.Rational(1,72)

w = [ sp.Rational(4,9) ] + [ sp.Rational(1,9) ]*4 + [ sp.Rational(1,36) ]*4

ww =  [ [[0]*2]*2 ] \
    + [ [ [  -one18, 0 ], [ 0, 2*one18 ] ] ]*2 \
    + [ [ [ 2*one18, 0 ], [ 0,  -one18 ] ] ]*2 \
    + [ [ [ -one72, -one12 ], [ -one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72, -one12 ], [ -one12, -one72 ] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(2))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(2) ]) \
        + sum([ ww[i][j][(j+1)%2]/cs**2*G[j,(j+1)%2] for j in range(1) ])
    return f

feq = sp.Matrix([ f(i) for i in range(Q) ])
feq[0] = rho - sum(feq[1:])

mf = sp.simplify(M@feq)

In [56]:
k = sp.symbols('k')
rho0 = sp.symbols('rho_0')
lb,ls,lq,le = sp.symbols('lambda_b, lambda_s, lambda_q, lambda_epsilon')
d = 6*(sp.symbols('c_s')**2 + rho0*kappa*k**2 - cs**2)
Omega = sp.zeros(Q,Q)
Omega[3,3] = lb
Omega[4,4] = ls
Omega[6,6] = lq
Omega[8,8] = le
Omega[5,5] = Omega[4,4]
Omega[7,7] = Omega[6,6]
Omega[3,0] = -lb*d
Omega[8,0] = le*d

In [57]:
kappa = sp.symbols('kappa')
pb = sp.symbols('p_b')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
kx, ky, kz = sp.symbols('k_x, k_y, k_z')
ks = {x:kx,y:ky,z:kz}
u = sp.Matrix([ux,uy])
Drho2 = sp.Matrix([ [ 0 for b in [x,y] ] for a in [x,y] ])
D2rho = -k**2*rho
G = kappa*Drho2

one12 = sp.Rational(1,12)
one18 = sp.Rational(1,18)
one72 = sp.Rational(1,72)

w = [ sp.Rational(4,9) ] + [ sp.Rational(1,9) ]*4 + [ sp.Rational(1,36) ]*4

ww =  [ [[0]*2]*2 ] \
    + [ [ [  -one18, 0 ], [ 0, 2*one18 ] ] ]*2 \
    + [ [ [ 2*one18, 0 ], [ 0,  -one18 ] ] ]*2 \
    + [ [ [ -one72, -one12 ], [ -one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72, -one12 ], [ -one12, -one72 ] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(2))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(2) ]) \
        + sum([ ww[i][j][(j+1)%2]/cs**2*G[j,(j+1)%2] for j in range(1) ])
    return f

feq = sp.Matrix([ f(i) for i in range(Q) ])
feq[0] = rho - sum(feq[1:])

In [58]:
fbar = feq.subs({pb:rho*sp.symbols('c_s')**2}).subs({rho:rho0,ux:0,uy:0})
S = sp.symbols('S', cls=sp.Function)(k)/rho0*sp.diag(*fbar)
G = sp.simplify(M@S@M.T)
G

Matrix([
[                                       S(k),                                0,                                0, 2*(3*c_s**2 + 3*k**2*kappa*rho_0 - 1)*S(k),                                    0,                                  0,                                  0,                                  0, 2*(-3*c_s**2 - 3*k**2*kappa*rho_0 + 1)*S(k)],
[                                          0, (c_s**2 + k**2*kappa*rho_0)*S(k),                                0,                                          0,                                    0,                                  0,                                  0,                                  0,                                           0],
[                                          0,                                0, (c_s**2 + k**2*kappa*rho_0)*S(k),                                          0,                                    0,                                  0,                                  0,                        

In [59]:
Lambda = sp.eye(Q)+Omega
Xi = sp.simplify(G-Lambda@G@Lambda.T)
Xi

Matrix([
[0, 0, 0,                                                                                                                                                                                         0,                                                            0,                                                          0,                                                          0,                                                          0,                                                                                                                                                                                                                                            0],
[0, 0, 0,                                                                                                                                                                                         0,                                                            0,                                                       

# D3Q19 model

## Multiphase model

### Basis transform matrix

In [60]:
d3q19 = LBStencil(Stencil.D3Q19)
D = len(d3q19[0])
Q = len(d3q19)
c = sp.Matrix(d3q19)
cs = sp.sqrt(sp.Rational(1,3))

x, y, z = MOMENT_SYMBOLS
one = sp.core.sympify(1)
c2 = x**2+y**2+z**2
c4 = c2**2

moments = [ 
    one,
    x,
    y,
    z,
    c2-1,
    3*x**2-c2,
    y**2-z**2,
    x*y,
    y*z,
    z*x,
    (3*c2-5)*x,
    (3*c2-5)*y,
    (3*c2-5)*z,
    (y**2-z**2)*x,
    (z**2-x**2)*y,
    (x**2-y**2)*z,
    3*c4-6*c2+1,
    (2*c2-3)*(3*x**2-c2),
    (2*c2-3)*(y**2-z**2)
]

M = moment_matrix(moments, stencil=d3q19)

### Setting derivates to correct values in Swift paper

Calculation of correct mf and feq

In [61]:
kappa = sp.symbols('kappa')
pb = sp.symbols('p_b')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
kx, ky, kz = sp.symbols('k_x, k_y, k_z')
ks = {x:kx,y:ky,z:kz}
u = sp.Matrix([ux,uy,uz])
Drho2 = sp.Matrix([ [ sp.Derivative(rho,a)*sp.Derivative(rho,b) for b in [x,y,z] ] for a in [x,y,z] ])
D2rho = sum([ sp.Derivative(rho,a,2) for a in [x,y,z] ])
G = kappa*Drho2

five36 = sp.Rational(5,36)
one9 = sp.Rational(1,9)
one12 = sp.Rational(1,12)
one36 = sp.Rational(1,36)
one72 = sp.Rational(1,72)

w = [ sp.Rational(1,3) ] + [ sp.Rational(1,18) ]*6 + [ sp.Rational(1,36) ]*12

ww =  [ [[0]*3]*3 ] \
    + [ [ [ -one9, 0, 0], [0, five36, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [five36, 0, 0], [0,  -one9, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [ -one9, 0, 0], [0,  -one9, 0], [0, 0, five36] ] ]*2 \
    + [ [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(D))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(D) ]) \
        + sum([ ww[i][j][(j+1)%D]/cs**2*G[j,(j+1)%D] for j in range(D) ])
    return f

feq = sp.Matrix([ f(i) for i in range(0,Q) ])
feq[0] = rho - sum(feq[1:])
feq.simplify()
mf = sp.simplify(M*feq)

In [62]:
rho0 = sp.symbols("rho0")
k = sp.symbols('k')
fbar = feq.subs({pb:rho*sp.symbols('c_s')**2}).subs({rho:rho0,ux:0,uy:0,uz:0})
S = sp.symbols('S', cls=sp.Function)(k)/rho0*sp.diag(*fbar)
G = sp.simplify(M@S@M.T)
G

Matrix([
[                                   S(k),           0,           0,           0,  (6*c_s**2*rho0 - 2*rho0)*S(k)/(2*rho0),             0,               0,             0,             0,             0,             0,             0,             0,               0,               0,               0, (-6*c_s**2*rho0 + 2*rho0)*S(k)/(2*rho0),             0,               0],
[                                      0, c_s**2*S(k),           0,           0,                                       0,             0,               0,             0,             0,             0,             0,             0,             0,               0,               0,               0,                                       0,             0,               0],
[                                      0,           0, c_s**2*S(k),           0,                                       0,             0,               0,             0,             0,             0,             0,             0,             0,          

### Setting derivates to 0

Assumes at k $\rightarrow$ 0 limit

In [63]:
kappa = sp.symbols('kappa')
pb = sp.symbols('p_b')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
kx, ky, kz = sp.symbols('k_x, k_y, k_z')
ks = {x:kx,y:ky,z:kz}
u = sp.Matrix([ux,uy,uz])
Drho2 = sp.Matrix([ [ 0 for b in [x,y,z] ] for a in [x,y,z] ])
D2rho = -k**2*rho
G = kappa*Drho2

five36 = sp.Rational(5,36)
one9 = sp.Rational(1,9)
one12 = sp.Rational(1,12)
one36 = sp.Rational(1,36)
one72 = sp.Rational(1,72)

w = [ sp.Rational(1,3) ] + [ sp.Rational(1,18) ]*6 + [ sp.Rational(1,36) ]*12

ww =  [ [[0]*3]*3 ] \
    + [ [ [ -one9, 0, 0], [0, five36, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [five36, 0, 0], [0,  -one9, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [ -one9, 0, 0], [0,  -one9, 0], [0, 0, five36] ] ]*2 \
    + [ [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(D))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(D) ]) \
        + sum([ ww[i][j][(j+1)%D]/cs**2*G[j,(j+1)%D] for j in range(D) ])
    return f

feq = sp.Matrix([ f(i) for i in range(0,Q) ])
feq[0] = rho - sum(feq[1:])
feq.simplify()
mf = sp.simplify(M*feq)

In [64]:
mf

Matrix([
[                                                                                                         rho(x, y, z)],
[                                                                                                     u_x*rho(x, y, z)],
[                                                                                                     u_y*rho(x, y, z)],
[                                                                                                     u_z*rho(x, y, z)],
[3*k**2*kappa*rho(x, y, z)**2 + 3*p_b + u_x**2*rho(x, y, z) + u_y**2*rho(x, y, z) + u_z**2*rho(x, y, z) - rho(x, y, z)],
[                                                                            (2*u_x**2 - u_y**2 - u_z**2)*rho(x, y, z)],
[                                                                                       (u_y**2 - u_z**2)*rho(x, y, z)],
[                                                                                                 u_x*u_y*rho(x, y, z)],
[                      

In [65]:
rho0 = sp.symbols("rho0")
k = sp.symbols('k')
fbar = feq.subs({pb:rho*sp.symbols('c_s')**2}).subs({rho:rho0,ux:0,uy:0,uz:0})
S = sp.symbols('S', cls=sp.Function)(k)/rho0*sp.diag(*fbar)
G = sp.simplify(M@S@M.T)
G

Matrix([
[                                    S(k),                               0,                               0,                               0, (3*c_s**2 + 3*k**2*kappa*rho0 - 1)*S(k),                                 0,                                   0,                                 0,                                 0,                                 0,                                 0,                                 0,                                 0,                                   0,                                   0,                                   0, (-3*c_s**2 - 3*k**2*kappa*rho0 + 1)*S(k),                                 0,                                   0],
[                                       0, (c_s**2 + k**2*kappa*rho0)*S(k),                               0,                               0,                                       0,                                 0,                                   0,                                 0,           

In [66]:
lb,ls,lq,le = sp.symbols('lambda_b, lambda_s, lambda_q, lambda_epsilon')
d = 6*(sp.symbols('c_s')**2 + rho0*kappa*k**2 - cs**2)
Omega = sp.zeros(Q,Q)

Omega[4,4]   = lb
Omega[5,5]   = lb
Omega[6,6]   = lb
Omega[7,7]   = ls
Omega[8,8]   = ls
Omega[9,9]   = ls
Omega[10,10] = lq
Omega[11,11] = lq
Omega[12,12] = lq
Omega[13,13] = lq
Omega[14,14] = lq
Omega[15,15] = lq
Omega[16,16] = le
Omega[17,17] = lq
Omega[18,18] = lq

Omega[4,0] = d/2*lb
Omega[16,0] = -d/2*le
Omega[16,4] = d/2*le

Omega

Matrix([
[                                                 0, 0, 0, 0,                                                 0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0,              0,        0,        0],
[                                                 0, 0, 0, 0,                                                 0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0,              0,        0,        0],
[                                                 0, 0, 0, 0,                                                 0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0,              0,        0,        0],
[                                                 0, 0, 0, 0,                                                 0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0

In [67]:
Lambda = sp.eye(Q)+Omega
Xi = sp.simplify(G-Lambda@G@Lambda.T)
Xi

Matrix([
[                                                                                       0, 0, 0, 0,                                                                                                                                                                                                                                                             2*lambda_b*(-3*c_s**2 - 3*k**2*kappa*rho0 + 1)*S(k),                                                         0,                                                           0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                           0,                                                    

## Multicomponent model

In [68]:
d3q19 = LBStencil(Stencil.D3Q19)
Q = len(d3q19)
c = sp.Matrix(d3q19)
cs = sp.sqrt(sp.Rational(1,3))

x, y, z = MOMENT_SYMBOLS
one = sp.core.sympify(1)
c2 = x**2+y**2+z**2
c4 = c2**2

moments = [ 
    one,
    x,
    y,
    z,
    c2-1,
    3*x**2-c2,
    y**2-z**2,
    x*y,
    y*z,
    z*x,
    (3*c2-5)*x,
    (3*c2-5)*y,
    (3*c2-5)*z,
    (y**2-z**2)*x,
    (z**2-x**2)*y,
    (x**2-y**2)*z,
    3*c4-6*c2+1,
    (2*c2-3)*(3*x**2-c2),
    (2*c2-3)*(y**2-z**2)
]

M = moment_matrix(moments, stencil=d3q19)

In [69]:
propagator = sp.diag(*(-sp.I*c@sp.Matrix([kx,ky,kz])).applyfunc(sp.exp))
propconj = sp.diag(*(+sp.I*c@sp.Matrix([kx,ky,kz])).applyfunc(sp.exp))
assert(sp.simplify(propagator@propconj)==sp.eye(Q))

### Equilibrium distributions

In [70]:
kappa = sp.symbols('kappa')
Gamma = sp.symbols('Gamma_phi')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
phi = sp.symbols('phi',cls=sp.Function)(x,y,z)
pb = sp.symbols('p_b')
mu = sp.symbols('mu_phi')
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
u = sp.Matrix([ux,uy,uz])

D2rho = sum([ sp.Derivative(rho,a,2) for a in [x,y,z] ])
D2phi = sum([ sp.Derivative(phi,a,2) for a in [x,y,z] ])
Drho2 = sp.Matrix([ [ sp.Derivative(rho,a)*sp.Derivative(rho,b) for b in [x,y,z] ] for a in [x,y,z] ])
Dphi2 = sp.Matrix([ [ sp.Derivative(phi,a)*sp.Derivative(phi,b) for b in [x,y,z] ] for a in [x,y,z] ])
G = kappa*Drho2+kappa*Dphi2

Gxx, Gyy, Gzz, Gxy, Gyz, Gzx = sp.symbols('G_xx G_yy G_zz G_xy G_yz G_zx')
G = sp.Matrix([[Gxx,Gxy,Gzx],[Gxy,Gyy,Gyz],[Gzx,Gyz,Gzz]])

five36 = sp.Rational(5,36)
one9 = sp.Rational(1,9)
one12 = sp.Rational(1,12)
one36 = sp.Rational(1,36)
one72 = sp.Rational(1,72)

w = [ sp.Rational(1,3) ] + [ sp.Rational(1,18) ]*6 + [ sp.Rational(1,36) ]*12

ww =  [ [[0]*3]*3 ] \
    + [ [ [ -one9, 0, 0], [0, five36, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [five36, 0, 0], [0,  -one9, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [ -one9, 0, 0], [0,  -one9, 0], [0, 0, five36] ] ]*2 \
    + [ [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(3))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho+kappa*phi*D2phi) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(3) ]) \
        + sum([ ww[i][j][(j+1)%3]/cs**2*G[j,(j+1)%3] for j in range(3) ])
    return f

feq = sp.Matrix([ f(i) for i in range(0,Q) ])
feq[0] = rho - sum(feq[1:])

def g(i):
    g = w[i]/cs**2*Gamma*mu \
        + w[i]/cs**2*phi*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*phi*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(3))@u)
    return g

geq = sp.Matrix([ g(i) for i in range(0,Q) ])
geq[0] = phi - sum(geq[1:])

In [71]:
mf = sp.simplify(M@feq)
mf

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                      rho(x, y, z)],
[                                                                                                                                                                                                                                                                                                                                                                                                                                                  u_x*rho(x, y, z)],
[                                                                                  

In [72]:
mg = sp.simplify(M@geq)
mg

Matrix([
[                                                                                       phi(x, y, z)],
[                                                                                   u_x*phi(x, y, z)],
[                                                                                   u_y*phi(x, y, z)],
[                                                                                   u_z*phi(x, y, z)],
[3*Gamma_phi*mu_phi + u_x**2*phi(x, y, z) + u_y**2*phi(x, y, z) + u_z**2*phi(x, y, z) - phi(x, y, z)],
[                                                          (2*u_x**2 - u_y**2 - u_z**2)*phi(x, y, z)],
[                                                                     (u_y**2 - u_z**2)*phi(x, y, z)],
[                                                                               u_x*u_y*phi(x, y, z)],
[                                                                               u_y*u_z*phi(x, y, z)],
[                                                               

In [73]:
D2rho = -k**2*rho
D2phi = -k**2*phi
Drho2 = sp.Matrix([ [ 0 for b in [x,y,z] ] for a in [x,y,z] ])
Dphi2 = sp.Matrix([ [ 0 for b in [x,y,z] ] for a in [x,y,z] ])
G = kappa*Drho2+kappa*Dphi2

feq = sp.Matrix([ f(i) for i in range(0,Q) ])
feq[0] = rho - sum(feq[1:])

geq = sp.Matrix([ g(i) for i in range(0,Q) ])
geq[0] = phi - sum(geq[1:])

In [74]:
chi = sp.symbols('chi')
T = sp.symbols('T')
k = sp.symbols('k')
rho0 = sp.symbols('rho_0')
phi0 = sp.symbols('phi_0')
fb = chi/4*rho*(1-phi**2/rho**2)-T*rho+T/2*(rho+phi)*sp.ln((rho+phi)/2)+T/2*(rho-phi)*sp.ln((rho-phi)/2)
mu = sp.simplify(sp.Derivative(fb,phi,1).doit())+kappa*k**2*phi
mu_rho = sp.collect(sp.Derivative(mu,rho,1).doit(),[chi,T,kappa]).subs({rho:rho0,phi:phi0})
mu_phi = sp.collect(sp.Derivative(mu,phi,1).doit(),[chi,T,kappa]).subs({rho:rho0,phi:phi0})

### Collision matrix

In [75]:
lb,ls,lo,le = sp.symbols('lambda_b, lambda_s, lambda_o, lambda_e')
sigma_s = sp.symbols('sigma_s')
dmf =  chi/2*phi0/rho0**2 - T*phi0/(rho0**2-phi0**2)
dmg = -chi/2/rho0 + T*rho0/(rho0**2-phi0**2) + kappa*k**2
df = 3*(sp.symbols('c_s')**2 + rho0*kappa*k**2 - sigma_s**2)
dg = 3*(Gamma*dmg - sigma_s**2)
Omega = sp.zeros(2*Q,2*Q)
for q in [0,Q]:
    Omega[q+4,q+4] = lb
    for i in range(q+5,q+10): Omega[i,i] = ls
    for i in range(q+10,q+16): Omega[i,i] = lo
    for i in range(q+16,q+Q): Omega[i,i] = le
Omega[4,0]    = -lb*df
Omega[16,0]   =  le*df
Omega[Q+4,Q]  = -lb*dg
Omega[Q+16,Q] =  le*dg
Omega[4,Q]    = -lb*3*phi0*kappa*k**2
Omega[16,Q]   =  le*3*phi0*kappa*k**2
Omega[Q+4,0]  = -lb*3*Gamma*dmf
Omega[Q+16,0] =  le*3*Gamma*dmf
assert(sp.simplify(Omega[Q+4,0]/lb-(-3*Gamma*mu_rho))==0)
assert(sp.simplify(Omega[Q+4,Q]/lb-(-3*Gamma*(mu_phi-sigma_s**2/Gamma)))==0)

### Correlations

In [76]:
fbar = feq.subs({pb:rho*sp.symbols('c_s')**2}).subs({rho:rho0,phi:phi0,ux:0,uy:0,uz:0})
gbar = geq.subs({pb:rho*sp.symbols('c_s')**2}).subs({rho:rho0,phi:phi0,ux:0,uy:0,uz:0})

S_rr = sp.Function('S_rho_rho')(k)/rho0*sp.diag(*fbar)
S_pp = sp.Function('S_phi_phi')(k)/phi0*sp.diag(*gbar)
S_rp = sp.zeros(Q)

S = sp.Matrix([[S_rr,S_rp],[S_rp,S_pp]])

In [77]:
N = sp.Matrix([[M,sp.zeros(Q)],[sp.zeros(Q),M]])
G = sp.simplify(N@S@N.T)

In [78]:
Lambda = sp.eye(2*Q)+Omega
Xi = sp.simplify(G-Lambda@G@Lambda.T)

In [18]:
G[:10,:10]

Matrix([
[                                    S_rho_rho(k),                                        0,                                        0,                                        0, (3*c_s**2 + 3*k**2*kappa*rho_0 - 1)*S_rho_rho(k),                                          0,                                            0,                                          0,                                          0,                                          0],
[                                               0, (c_s**2 + k**2*kappa*rho_0)*S_rho_rho(k),                                        0,                                        0,                                                0,                                          0,                                            0,                                          0,                                          0,                                          0],
[                                               0,                                        0

In [19]:
G[4,4]

(-c_s**2 - k**2*kappa*rho_0 + 1)*S_rho_rho(k)